# Context

## 分析合成控制结果

**Abstract:**
One-sentence description

**Description:**
In the following cell, I...


In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
from matplotlib import pyplot as plt

from src import NATURE_PALETTE as nature_colors

In [ ]:
from src.multi_synth import MultiSynth
from core.overall_analysis import add_statistic_items
import os
from hydra import compose, initialize


with initialize(version_base=None, config_path="../config"):
    os.chdir("../")
    cfg = compose(config_name="synth")

exp87 = MultiSynth.load_from_pickle("model/model_87.pkl")
exp98 = MultiSynth.load_from_pickle("model/model_98.pkl")
statistic_87 = add_statistic_items(exp87, cfg)
statistic_98 = add_statistic_items(exp98, cfg)

In [ ]:
from core.overall_analysis import analysis_exp

msg = analysis_exp(exp87)
print(msg)

In [ ]:
msg = analysis_exp(exp98)
print(msg)

In [ ]:
from core.plots import plot_comprehensive_fig1

plot_comprehensive_fig1(exp87, exp98, cfg, save="figs/outputs/main_results2.pdf")

In [ ]:
exp87.plot_panels("trend", figsize=(12, 6), save="figs/outputs/87placebo.pdf")
exp98.plot_panels("trend", figsize=(12, 6), save="figs/outputs/98placebo.pdf")

In [ ]:
exp87.plot_panels(save="figs/outputs/87rmse.pdf")
exp98.plot_panels(save="figs/outputs/98rmse.pdf")

In [ ]:
exp87.rmse_report()

使用合成控制法的安慰剂检验，使用均值+-两倍[标准差](https://www.zhihu.com/question/22864111)对显著性进行判断。

## Compare 87-98 regulating

**Abstract:**
One-sentence description

**Description:**
In the following cell, I...


In [ ]:
from src.plots import correlation_analysis

correlation_analysis(statistic_98, xs=["YR_WU", "satisfied"], y="diff_ratio")

In [ ]:
from core.overall_analysis import get_compare_df
from core.plots import comparison_plot

compare_df = get_compare_df(statistic_87, statistic_98)
comparison_plot(compare_df, save="figs/outputs/fig3.pdf")

In [ ]:
compare_df["87_ratio"].iloc[:4].mean()
compare_df["98_ratio"]

In [ ]:
compare_df["98_ratio"][compare_df["98_ratio"] < 0].mean()

In [ ]:
compare_df.loc["Neimeng", :]

In [ ]:
(
    compare_df.loc["Neimeng", "98_ratio"] - compare_df.loc["Neimeng", "87_ratio"]
) / compare_df.loc["Neimeng", "87_ratio"]